# Rachunek macierzowy - mnożenie macierzy

**Wykonali: Alicja Niewiadomska, Paweł Kruczkiewicz**


In [1]:
import numpy as np
import matplotlib.pyplot as plt


Wybrano temat nr 3, czyli *dla macierzy mniejszych lub równych 2^l x 2^l mnożenie rekurencyjne metodą Bineta, dla większych - Strassena*.


Wybrane algorytmy macierzowe:
 1. Mnożenie rekurencyjne metodą Bineta.
 2. Mnożenie rekurencyjne metodą Strassena.
 

## Pseudokod rozwiązania

Zakładamy, że dana macierz ma wymiary 2^k x 2^k.

Dane: 
  - `A`, `B` - macierze kwadratowe o rozmiarze 2^k
  - `k` - wyżej wspomniany wykładnik
  - `l` - arbitralnie wybrana eksponenta wartości progowej dla danego algorytmu.
  
Wartość zwracana: 
   `C` - wynikowa macierz kwadratowa o rozmiarze 2^k
```
def mat_mul(A, B, k, l):
    if l <= k:
        return binet_rec_mat_mul(A, B, k, l)
    else:
        return strass_mat_mul(A, B, k, l)
```

## Kod algorytmu

### Funkcje pomocnicze

In [ ]:
def split_matrix_in_quarters(A: np.array):
    half_i = A.shape[0] // 2
    return  A[:half_i, :half_i], \
            A[:half_i, half_i:], \
            A[half_i:, :half_i],\
            A[half_i:, half_i:]
            
def join_matrices(A11, A12, A21, A22):
    A1 = np.hstack((A11, A12))
    A2 = np.hstack((A21, A22))
    return np.vstack((A1, A2))

### Algorytm rekurencyjny Bineta

In [5]:
def binet_rec_mat_mul(A: np.array, B: np.array, k: int, l: int) -> np.array:
    if k == 0:
        return A*B
    
    rec_step = lambda A1, B1, A2, B2: mat_mul(A1, B1, k-1, l) + mat_mul(A2, B2, k-1, l)

    A11, A12, A21, A22 = split_matrix_in_quarters(A)
    B11, B12, B21, B22 = split_matrix_in_quarters(B)
    
    C11 = rec_step(A11, B11, A12, B21)
    C12 = rec_step(A11, B12, A12, B22)
    C21 = rec_step(A21, B11, A22, B21)
    C22 = rec_step(A21, B12, A22, B22)
    
    return join_matrices(C11, C12, C21, C22)



### Algorytm rekurencyjny Strassena

In [ ]:
def strassen_mat_mul(A, B, k, l):
    pass

### Algorytm końcowy

In [ ]:
def mat_mul(A, B, k, l):
    if l <= k:
        return binet_rec_mat_mul(A, B, k, l)
    else:
        return strassen_mat_mul(A, B, k, l)

## Wykresy
Wybrane wielkości parametru `l`: 4, 6, 8
### Wykres czasu mnożenia od wielkości macierzy

In [4]:
### kod generujący czas
### kod wykresu

### Wykres wykonanych obliczeń zmiennoprzecinkowych w zależności od wielkości macierzy

In [ ]:
### kod generujący liczbę obliczeń zmiennoprzecinkowych

def binet_rec_mat_mul_op_count(k: int, l: int) -> np.array:
    if k == 0:
        return 1  # multiplying matrices
     
    rec_step = lambda: mat_mul_op_count(k-1, l) + mat_mul_op_count(k-1, l) + 1 # recursive step plus one addition
    
    C11_op_count = rec_step()
    C12_op_count = rec_step()
    C21_op_count = rec_step()
    C22_op_count = rec_step()
    
    return C11_op_count + C12_op_count + C21_op_count + C22_op_count
    

### kod wykresu